In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from matplotlib import pyplot as plt
import math 
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Curso_DL_CFP/proyecto/datos_nc_modelos/dataset_codif.csv', on_bad_lines='skip', sep=',')
df.describe()

In [ ]:
# Box Plot
import seaborn as sns
sns.boxplot(df['likes'])

In [33]:
#Elimino post con más de 200000
data = df[df['likes']>= 200000].index
df.drop(data, inplace = True)

In [ ]:
from sklearn.model_selection import train_test_split

# We have to pass np arrays instead of pd dataframes or series to Keras: .values
#X = df[[ 'followers','sponsored', 'followees', 'nIgtv','business','verified', 'day', 'month', 'hour_of_day', 'category', 'label', 'is_people' ]].values
#X = df[[ 'followers','sponsored', 'followees', 'nIgtv','business','verified', 'day', 'month', 'hour_of_day', 'category' ]].values
#X = df[[ 'followers', 'nIgtv','business','verified', 'hour_of_day', 'label', 'is_people']].values
X = df[[ 'followers', 'nIgtv','business','verified', 'hour_of_day']].values
#X = df[[ 'followers', 'followees','verified', 'hour_of_day', 'day', 'category', 'month']].values
y = df['likes'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print( X_train.shape , X_test.shape)

REDES NEURONALES:

In [ ]:
from keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam 

from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
#from keras.models import Model
 
# define our MLP network
model1 = Sequential()
model1.add(Dense(12, input_dim =12, activation="relu"))
model1.add(Dense(14, activation="relu"))
model1.add(Dense(64, activation="relu"))
model1.add(Dense(64, activation="relu"))
model1.add(Dense(64, activation="relu"))
model1.add(Dense(64, activation="relu"))
model1.add(Dense(64, activation="relu"))
model1.add(Dense(64, activation="relu"))
model1.add(Dense(1, activation="linear"))
opt = Adam(lr=1e-4, decay=1e-4 / 200)
model1.compile(loss="mean_squared_error", optimizer=opt)

model1.fit(X_train, y_train, validation_data=(X_test, y_test),
	epochs=300, batch_size=100)

loss_df = pd.DataFrame(model1.history.history)
loss_df.plot()

In [67]:
model1.save("/content/drive/My Drive/Curso_DL_CFP/Modelos/deepANN_fin.h5")

In [ ]:
from math import sqrt 
test_predictions = model1.predict(X_test)
test_predictions = pd.Series(test_predictions.reshape(2684,))
pred_df = pd.DataFrame(y_test, columns=['Test True Y'])
pred_df = pd.concat([pred_df, test_predictions], axis=1)
pred_df.columns = ['Test True Y', 'Model Predictions']
rmse_la = sqrt(mean_squared_error(y_test, test_predictions)) 
Rsquared = r2_score(y_test, test_predictions)
mae = mean_absolute_error(y_test, test_predictions)
print('R Squared: %.3f' % Rsquared)
print('RMSE: %.3f' % rmse_la) 
print('MAE: %.3f' % mae)
print('-------------------------------------------------')
sns.scatterplot(x='Test True Y', y='Model Predictions', data=pred_df)
sns.regplot(x='Test True Y', y='Model Predictions', data=pred_df, color='purple' )

In [ ]:
pred_df.head(20)


REGRESIÓN 


In [7]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore') 
import math
import scipy
import plotly.graph_objects as go 
import plotly.express as px
from math import sqrt
from statistics import mean
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, balanced_accuracy_score 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, KFold 
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import datasets
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import xgboost as xgb

In [ ]:
model_baseline = KNeighborsRegressor()
model_baseline.fit(X_train, y_train)
prediction_engagement = model_baseline.predict(X_test)

#Performance Metrics (Errors)
rmse_la = sqrt(mean_squared_error(y_test, prediction_engagement)) 
Rsquared = r2_score(y_test, prediction_engagement)
mae = mean_absolute_error(y_test, prediction_engagement)
print('R Squared: %.3f' % Rsquared)
print('RMSE: %.3f' % rmse_la) 
print('MAE: %.3f' % mae)

#Plot actual vs predicted
fig, ax = plt.subplots()
ax.scatter(y_test, prediction_engagement, edgecolors=(0, 0, 0)) 
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()


In [72]:
models = [KNeighborsRegressor() ,DecisionTreeRegressor() ,GradientBoostingRegressor(), xgb.XGBRegressor() ]

In [ ]:
R2 = []
RMSE = []
MAE = []

for model in models:

  model.fit(X_train, y_train)
  prediction_engagement = model.predict(X_test)

  #Performance Metrics (Errors)
  rmse_la = sqrt(mean_squared_error(y_test, prediction_engagement)) 
  Rsquared = r2_score(y_test, prediction_engagement)
  mae = mean_absolute_error(y_test, prediction_engagement)
  print('R Squared: %.3f' % Rsquared)
  R2.append(Rsquared)
  print('RMSE: %.3f' % rmse_la)
  RMSE.append(rmse_la) 
  print('MAE: %.3f' % mae)
  MAE.append(mae)

  #Plot actual vs predicted
  fig, ax = plt.subplots()
  ax.scatter(y_test, prediction_engagement, edgecolors=(0, 0, 0)) 
  ax.set_xlabel('Measured')
  ax.set_ylabel('Predicted')
  plt.show()




In [ ]:
col={'R2':R2,'RMSE':RMSE, 'MAE': MAE}
models=['KNN','DTree','Gradient Boosting', 'XGBoost']
df=pd.DataFrame(data=col,index=models)
df

In [ ]:
model3 = KNeighborsRegressor(10) 
model3.fit(X_train, y_train)
prediction_engagement = model3.predict(X_test)

#Performance Metrics (Errors)
rmse_la = sqrt(mean_squared_error(y_test, prediction_engagement)) 
Rsquared = r2_score(y_test, prediction_engagement)
mae = mean_absolute_error(y_test, prediction_engagement)
print('R Squared: %.3f' % Rsquared)
print('RMSE: %.3f' % rmse_la) 
print('MAE: %.3f' % mae)

#Plot actual vs predicted
fig, ax = plt.subplots()
ax.scatter(y_test, prediction_engagement, edgecolors=(0, 0, 0)) 
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()


In [ ]:
model3 = xgb.XGBRegressor() 
model3.fit(X_train, y_train)
prediction_engagement = model3.predict(X_test)

#Performance Metrics (Errors)
rmse_la = sqrt(mean_squared_error(y_test, prediction_engagement)) 
Rsquared = r2_score(y_test, prediction_engagement)
mae = mean_absolute_error(y_test, prediction_engagement)
print('R Squared: %.3f' % Rsquared)
print('RMSE: %.3f' % rmse_la) 
print('MAE: %.3f' % mae)

#Plot actual vs predicted
fig, ax = plt.subplots()
ax.scatter(y_test, prediction_engagement, edgecolors=(0, 0, 0)) 
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')

plt.show()


In [ ]:
prediction_engagement = pd.Series(prediction_engagement.reshape(2829,))
pred_df = pd.DataFrame(y_test, columns=['Test True Y'])
pred_df = pd.concat([pred_df, prediction_engagement], axis=1)
pred_df.columns = ['Test True Y', 'Model Predictions']
pred = pred_df [pred_df['Model Predictions']>=35000]
pred

In [ ]:
pred_x = pd.DataFrame(X_test, columns=['followers', 'nIgtv','business','verified', 'hour_of_day'])
pred_df = pd.concat([pred_df, pred_x], axis=1)
pred_df.columns = ['Test True Y', 'Model Predictions', 'followers', 'nIgtv','business','verified', 'hour_of_day']
pred = pred_df [pred_df['Model Predictions']>=70000]
pred

In [ ]:
sns.scatterplot(x='Test True Y', y='Model Predictions', data=pred_df)
sns.regplot(x='Test True Y', y='Model Predictions', data=pred_df, color='purple' )

In [ ]:
model3 = GradientBoostingRegressor()
model3.fit(X_train, y_train)
prediction_engagement = model3.predict(X_test)

#Performance Metrics (Errors)
rmse_la = sqrt(mean_squared_error(y_test, prediction_engagement)) 
Rsquared = r2_score(y_test, prediction_engagement)
mae = mean_absolute_error(y_test, prediction_engagement)
print('R Squared: %.3f' % Rsquared)
print('RMSE: %.3f' % rmse_la) 
print('MAE: %.3f' % mae)

#Plot actual vs predicted
fig, ax = plt.subplots()
ax.scatter(y_test, prediction_engagement, edgecolors=(0, 0, 0)) 
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()
